# This is the Jupyter Notebook for the MAGICODE project

In [1]:
#!pip install -r requirements.txt
# fix issue ValueError: Object arrays cannot be loaded when allow_pickle=False
!pip uninstall numpy
!pip install numpy==1.16.2

^C


In [1]:
import os
os.chdir('model')

In [18]:
# split training set and evaluation set while ensuring no training example in the evaluation set
# usage: build_datasets.py <input path> <distribution (default: 2)>
%run -i build_datasets.py ../datasets/subset

Splitting datasets, training samples: 68.0, evaluation samples: 34.0
Training dataset: ../datasets/training_set
Evaluation dataset: ../datasets/eval_set


In [3]:
# transform images (normalized pixel values and resized pictures) in training dataset to numpy arrays (smaller files if you need to upload the set to train your model in the cloud)
# usage: convert_imgs_to_arrays.py <input path> <output path>
%run -i convert_imgs_to_arrays.py ../datasets/training_set ../datasets/training_features

Converting images to numpy arrays...
Numpy arrays saved in ../datasets/training_features


In [2]:
os.chdir('../')
if not os.path.exists('bin'):
    os.mkdir('bin')
os.chdir('model')

In [5]:
# train with generator to avoid having to fit all the data in memory (RECOMMENDED)
%run -i train.py ../datasets/training_features ../bin 1

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

Loading data...
Generating sparse vectors...
Dataset size: 1566
Vocabulary size: 50
Input shape: (256, 256, 3)
Output size: 50
Parsing data...
Epoch 1/10
25/24 [==============================] - 289s 12s/step - loss: 3.4243
Epoch 2/10
25/24 [==============================] - 285s 11s/step - loss: 3.3276
Epoch 3/10
25/24 [==============================] - 286s 11s/step - loss: 3.2976
Epoch 4/10
25/24 [==============================] - 283s 11s/step - loss: 3.2108
Epoch 5/10
25/24 [==============================] - 284s 11s/step - loss: 3.1496
Epoch 6/10
25/24 [==============================] - 286s 11s/step - loss: 3.1018
Epoch 7/10
25/24 [==============================] - 289s 12s/step - loss: 3.0449
Epoch 8/10
25/24 [==============================] - 283s 11s/step - loss: 3.0097
Epoch 9/10
25/24 [==============================] - 277s 11s/step - loss: 2.9533
Epoch 10/10
25/24 [=================

In [13]:
# generate DSL code (.gui file), the default search method is greedy
# usage: sample.py <trained weights path> <trained model name> <input image> <output path> <search method (default: greedy)>
%run -i sample.py ../bin pix2code ../test.png ../code

Vocabulary size: 50
Input shape: (256, 256, 3)
Output size: 50
Result greedy: <START><START><START><START>,{"element":"btn-warning"},{"element":"btn-warning"},"body":{"rows":[{"cards":[{"card":[{"element":"title"},"body":{"rows":[{"cards":[{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"},{"card":[{"element":"title"}<END>


In [ ]:
os.chdir('../compiler')
print(os.getcwd())

In [5]:
# compile .gui file to HTML/CSS (Bootstrap style)
%run -i web-compiler.py ../code/test.gui